---

# 1. Introduction
In this notebook, we will analyse the following betting strategies:
1. **Betting on the Favourite**(Team With Lower Odds)
2. **Betting on the Underdog**(Team with Higher Odds)
3. **Betting on a Draw**

For our purposes, we will merge the two tables from our SQLite database based off each fixture's unique `fixture_id` to form a table with the odds and match outcomes. We will then pick the best possible odds for our respective strategies for analysis.

---

# 2. Import Libraries
We import the necessary libraries for:
- Data manipulation using `pandas`.
- Interacting with the SQL database using `SQLAlchemy`.
- Generating and saving our visualisations with `lets_plot`.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from lets_plot import * 
from lets_plot import ggsave
from IPython.display import SVG

---

# 3. Merging Odds Data and Match Outcomes 

In [2]:
engine = create_engine('sqlite:///../data/sports_odds.db')
odds_df = pd.read_sql('SELECT * FROM historical_odds', con=engine)
results_df = pd.read_sql('SELECT * FROM match_results', con=engine)

After inspection, we realise that there is a different number of rows for `odds_df` and `results_df`. These discrepancies in the data size are normal and will not affect our analysis. Hence, we used `pandas` merge on 'inner' to ensure only matches with `fixture_id` present in both dataframes will be selected for analysis.

In [3]:
merged_df = odds_df.merge(results_df, on='fixture_id', how='inner')
merged_df.head()

,match_id,commence_time,home_team,away_team,Unibet_home_odds,Unibet_away_odds,Unibet_draw_odds,Sky Bet_home_odds,Sky Bet_away_odds,Sky Bet_draw_odds,...,Grosvenor_draw_odds,Smarkets_home_odds,Smarkets_away_odds,Smarkets_draw_odds,fixture_id,Date,Time,HomeTeam,AwayTeam,FTR
0,2dd4a4f8663e6f835226a5209c614a60,2020-06-17T17:00:00Z,Aston Villa,Sheffield United,3.35,2.32,3.25,3.10,2.25,3.30,...,NaN,NaN,NaN,NaN,AVLSHU170620,17/06/2020,18:00,Aston Villa,Sheffield United,D
1,b1e029a0d989b4c11e843204003044f9,2020-06-17T19:15:00Z,Manchester City,Arsenal,1.35,8.50,5.60,1.36,7.50,5.25,...,NaN,NaN,NaN,NaN,MCIARS170620,17/06/2020,20:15,Man City,Arsenal,H
2,59d68295dc2213634772cd941c91fa11,2020-06-19T19:15:00Z,Tottenham Hotspur,Manchester United,2.75,2.60,3.30,2.70,2.50,3.40,...,NaN,NaN,NaN,NaN,TOTMUN190620,19/06/2020,20:15,Tottenham,Man United,D
3,88352746f45f6beb4e2cb662d9414d0f,2020-06-20T11:30:00Z,Watford,Leicester City,3.40,2.15,3.45,3.25,2.20,3.40,...,NaN,NaN,NaN,NaN,WATLEI200620,20/06/2020,12:30,Watford,Leicester,D
4,065ae59da20562892de52b7f5598ecbf,2020-06-20T16:30:00Z,West Ham United,Wolverhampton Wanderers,3.50,2.15,3.35,3.30,2.20,3.30,...,NaN,NaN,NaN,NaN,WHUWOL200620,20/06/2020,17:30,West Ham,Wolves,A


---

# 4. Extracting the Highest Odds for Each Fixture and Outcome

We understand that getting the best odds for each fixture/outcome may be theoretically difficult as a retail better. However, this layer of analysis gives us further information about which bookmakers could potentially offer better odds for certain teams/outcomes and gives us more accuracy in predicting the profitability of our strategy.

In [4]:
# Import the function to process each row and obtain highest odds
from functions import process_row

# Apply the function to each row
new_columns = merged_df.apply(process_row, axis=1)
merged_df = pd.concat([merged_df, new_columns], axis=1)
merged_df

,match_id,commence_time,home_team,away_team,Unibet_home_odds,Unibet_away_odds,Unibet_draw_odds,Sky Bet_home_odds,Sky Bet_away_odds,Sky Bet_draw_odds,...,Time,HomeTeam,AwayTeam,FTR,max_home,home_bookmaker,max_away,away_bookmaker,max_draw,draw_bookmaker
0,2dd4a4f8663e6f835226a5209c614a60,2020-06-17T17:00:00Z,Aston Villa,Sheffield United,3.35,2.32,3.25,3.10,2.25,3.30,...,18:00,Aston Villa,Sheffield United,D,3.35,Unibet,2.41,Marathon Bet,3.52,Marathon Bet
1,b1e029a0d989b4c11e843204003044f9,2020-06-17T19:15:00Z,Manchester City,Arsenal,1.35,8.50,5.60,1.36,7.50,5.25,...,20:15,Man City,Arsenal,H,1.39,Marathon Bet,8.70,Marathon Bet,5.95,Marathon Bet
2,59d68295dc2213634772cd941c91fa11,2020-06-19T19:15:00Z,Tottenham Hotspur,Manchester United,2.75,2.60,3.30,2.70,2.50,3.40,...,20:15,Tottenham,Man United,D,2.88,Betfair,2.64,Marathon Bet,3.70,Marathon Bet
3,88352746f45f6beb4e2cb662d9414d0f,2020-06-20T11:30:00Z,Watford,Leicester City,3.40,2.15,3.45,3.25,2.20,3.40,...,12:30,Watford,Leicester,D,3.52,Marathon Bet,2.22,Marathon Bet,3.75,Marathon Bet
4,065ae59da20562892de52b7f5598ecbf,2020-06-20T16:30:00Z,West Ham United,Wolverhampton Wanderers,3.50,2.15,3.35,3.30,2.20,3.30,...,17:30,West Ham,Wolves,A,3.70,Betfair,2.23,Marathon Bet,3.60,Marathon Bet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,ac9d59dd7555c122948f98b8b2a19c8e,2024-12-08T14:00:00Z,Fulham,Arsenal,NaN,NaN,NaN,6.25,1.44,4.33,...,14:00,Fulham,Arsenal,D,6.40,Smarkets,1.62,BoyleSports,4.33,Sky Bet
1778,8899ce68b0c9e451b17edc3f2e076a6b,2024-12-08T14:00:00Z,Ipswich Town,Bournemouth,NaN,NaN,NaN,3.10,2.15,3.50,...,14:00,Ipswich,Bournemouth,A,3.20,Paddy Power,2.20,Smarkets,3.90,Betfair
1779,d9163188e3e3d9a8bf2525e2f9e3a553,2024-12-08T14:00:00Z,Leicester City,Brighton and Hove Albion,NaN,NaN,NaN,4.75,1.62,4.20,...,14:00,Leicester,Brighton,D,5.00,Paddy Power,1.73,BoyleSports,4.40,Coral
1780,c87f3d3551a57bd560cb056ade831890,2024-12-08T16:30:00Z,Tottenham Hotspur,Chelsea,NaN,NaN,NaN,2.20,2.80,3.75,...,16:30,Tottenham,Chelsea,A,2.30,Paddy Power,3.10,Smarkets,3.75,Sky Bet


In [5]:
merged_df = merged_df[['fixture_id', 'Date', 'home_team', 'away_team', 'FTR', 'max_home', 'home_bookmaker', 'max_away', 'away_bookmaker', 'max_draw', 'draw_bookmaker']]
merged_df

,fixture_id,Date,home_team,away_team,FTR,max_home,home_bookmaker,max_away,away_bookmaker,max_draw,draw_bookmaker
0,AVLSHU170620,17/06/2020,Aston Villa,Sheffield United,D,3.35,Unibet,2.41,Marathon Bet,3.52,Marathon Bet
1,MCIARS170620,17/06/2020,Manchester City,Arsenal,H,1.39,Marathon Bet,8.70,Marathon Bet,5.95,Marathon Bet
2,TOTMUN190620,19/06/2020,Tottenham Hotspur,Manchester United,D,2.88,Betfair,2.64,Marathon Bet,3.70,Marathon Bet
3,WATLEI200620,20/06/2020,Watford,Leicester City,D,3.52,Marathon Bet,2.22,Marathon Bet,3.75,Marathon Bet
4,WHUWOL200620,20/06/2020,West Ham United,Wolverhampton Wanderers,A,3.70,Betfair,2.23,Marathon Bet,3.60,Marathon Bet
...,...,...,...,...,...,...,...,...,...,...,...
1777,FULARS081224,08/12/2024,Fulham,Arsenal,D,6.40,Smarkets,1.62,BoyleSports,4.33,Sky Bet
1778,IPSBOU081224,08/12/2024,Ipswich Town,Bournemouth,A,3.20,Paddy Power,2.20,Smarkets,3.90,Betfair
1779,LEIBHA081224,08/12/2024,Leicester City,Brighton and Hove Albion,D,5.00,Paddy Power,1.73,BoyleSports,4.40,Coral
1780,TOTCHE081224,08/12/2024,Tottenham Hotspur,Chelsea,A,2.30,Paddy Power,3.10,Smarkets,3.75,Sky Bet


---

# 5. EDA: Which Bookmakers Offer the Best Odds?
First, let us look at the prevalence of bookmakers in providing the best odds. This gives us some insight into which bookmakers we can focus on when deciding to place bets, streamlining the process.

In [6]:
LetsPlot.setup_html()

In [7]:
# Function to get top bookmakers
def get_top_bookmakers(dataframe, column):
    
    total_matches = len(dataframe)
    counts = dataframe[column].value_counts(normalize=True) * 100

    # Get the top bookmakers
    top_counts = counts.nlargest(5).reset_index()
    top_counts.columns = ['Bookmaker', 'Percentage']
    
    return top_counts

top_home = get_top_bookmakers(merged_df, 'home_bookmaker')
top_away = get_top_bookmakers(merged_df, 'away_bookmaker')
top_draw = get_top_bookmakers(merged_df, 'draw_bookmaker')

In [8]:
top_home_bookmakers = (
    ggplot(top_home, aes(x='Bookmaker', y='Percentage'))
    + geom_bar(stat='identity', fill='#024B04')
    + ggsize(800, 400)
    + labs(
        title='Marathon Bet gives the highest home odds almost 20% of the time!',
        subtitle='The top five bookmakers provide the best odds almost 60% of the time!'
    )
    + theme(
        axis_text_x=element_text(size=12, angle=0, hjust=1),
        plot_title=element_text(face='bold', hjust=0.5, size=22),
        plot_subtitle=element_text(size=16, hjust=0.5, color='blue')
    )
)
top_home_bookmakers


In [9]:
top_away_bookmakers = (
    ggplot(top_away, aes(x='Bookmaker', y='Percentage'))
    + geom_bar(stat='identity', fill='#8B2E01')
    + ggsize(800, 400)
    + labs(
        title='Marathon Bet gives the highest away odds again!',
        subtitle='Similarly, the top five bookmakers provide the best away odds almost 60% of the time'
    )
    + theme(
        axis_text_x=element_text(size=12, angle=0, hjust=1),
        plot_title=element_text(face='bold', hjust=0.5, size=22),
        plot_subtitle=element_text(size=16, hjust=0.5, color='blue')
    )
)
top_away_bookmakers

In [10]:
top_draw_bookmakers = (
    ggplot(top_draw, aes(x='Bookmaker', y='Percentage'))
    + geom_bar(stat='identity', fill='grey')
    + ggsize(800, 400)
    + labs(
        title='Marathon Bet gives the highest odds almost 30% of the time, our clear winner!',
        subtitle='Sky Bet and Virgin Bet are additional contenders for good draw odds'
    )
    + theme(
        axis_text_x=element_text(size=12, angle=0, hjust=1),
        plot_title=element_text(face='bold', hjust=0.5, size=20),
        plot_subtitle=element_text(size=16, hjust=0.5, color='blue')
    )
)
top_draw_bookmakers

In [11]:
ggsave(top_home_bookmakers,"top_home_bookmakers.svg", path = "../data/visualisations/simple_betting_strategies", dpi = 300)
ggsave(top_away_bookmakers,"top_away_bookmakers.svg", path = "../data/visualisations/simple_betting_strategies", dpi = 300)
ggsave(top_draw_bookmakers,"top_draw_bookmakers.svg", path = "../data/visualisations/simple_betting_strategies", dpi = 300)

'c:\\Users\\Xinyan\\Desktop\\DS105A\\ds105a-2024-project-good_gamblers\\data\\visualisations\\simple_betting_strategies\\top_draw_bookmakers.svg'

**Conclusion**: Marathon Bet has the highest probability of providing the best odds for any particular outcome (Home, Away, Draw). This is unsurprising given its reputation as a low-margin bookmaker in the industry. Other strong contenders to consider are Paddy Power, Betclic, William Hill and Unibet.